In [ ]:
from collections import OrderedDict
import re
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from tqdm.notebook import tqdm
import numpy as np

from eval import get_run_metrics, read_run_dir, get_model_from_run
from plot_utils import basic_plot, collect_results, relevant_model_names

%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set_theme('notebook', 'darkgrid')
palette = sns.color_palette('colorblind')

run_dir = "../models"

In [ ]:
df = read_run_dir(run_dir)
df  # list all the runs in our run_dir

In [ ]:
task = "linear_regression"
#task = "sparse_linear_regression"
# task = "decision_tree"
#task = "relu_2nn_regression"

run_id = "toy_w1_1"  # if you train more models, replace with the run_id from the table above

run_path = os.path.join(run_dir, task, run_id)
recompute_metrics = False

if recompute_metrics:
    get_run_metrics(run_path)  # these are normally precomputed at the end of training

# Plot pre-computed metrics

In [ ]:
def valid_row(r):
    return r.task == task and r.run_id == run_id

metrics = collect_results(run_dir, df, valid_row=valid_row)
_, conf = get_model_from_run(run_path, only_conf=True)
n_dims = conf.model.n_dims

models = relevant_model_names[task]
basic_plot(metrics["standard"], models=models)
plt.show()

# Interactive setup

We will now directly load the model and measure its in-context learning ability on a batch of random inputs. (In the paper we average over multiple such batches to obtain better estimates.)

In [ ]:
from samplers import get_data_sampler
from tasks import get_task_sampler
print(conf.training.curriculum.points.end)

In [ ]:
model, conf = get_model_from_run(run_path)

n_dims = conf.model.n_dims
batch_size = conf.training.batch_size
data_sampler = get_data_sampler(conf.training.data, n_dims)
task_sampler = get_task_sampler(
    conf.training.task,
    n_dims,
    batch_size,
    **conf.training.task_kwargs
)

task = task_sampler()
xs = data_sampler.sample_xs(b_size=batch_size, n_points=conf.training.curriculum.points.end)
ys = task.evaluate(xs)
with torch.no_grad():
    pred = model(xs, ys)
metric = task.get_metric()
loss = metric(pred, ys).numpy()

sparsity = conf.training.task_kwargs.sparsity if "sparsity" in conf.training.task_kwargs else None
baseline = {
    "linear_regression": n_dims,
    "sparse_linear_regression": sparsity,
    "relu_2nn_regression": n_dims,
    "decision_tree": 1,
}[conf.training.task]

plt.plot(loss.mean(axis=0), lw=2, label="Transformer")
plt.axhline(baseline, ls="--", color="gray", label="zero estimator")
plt.xlabel("# in-context examples")
plt.ylabel("squared error")
plt.legend()
plt.show()

### 4.1 Sample Selection / Covariate Shifts

In [ ]:
def R_Square_Error(ys, pred):
    # Step 2: Calculate the mean of the actual outcomes
    y_mean = torch.mean(ys)

    # Step 3: Compute SS_tot and SS_res
    SS_tot = torch.sum((ys - y_mean) ** 2)
    SS_res = torch.sum((ys - pred) ** 2)

    # Step 4: Calculate R^2
    R_square = 1 - SS_res / SS_tot
    return R_square

####  Part 1 - Sample Selection: W1 = 1, w2 = 0

In [8]:
n_batches = 50
prompt_length = 2*conf.training.curriculum.points.end-1
# # save xs and ys for the 4.2 experiment
xs_list = [] 
ys_list = [] 
actual_points_w_1 = [[] for _ in range(prompt_length)]
predicted_points_w_1 = [[] for _ in range(prompt_length)]
# Generate data and perform the experiment
for _ in tqdm(range(n_batches)):

    xs = data_sampler.sample_xs(b_size=batch_size, n_points=prompt_length)
    # 64 x 101 x 20
    xs_list.append(xs)
    ys = task.evaluate(xs)
    # 64 x 101
    ys_list.append(ys)

    with torch.no_grad():
        pred = model(xs, ys)
    for j in range(prompt_length):
        actual_points_w_1[j].extend(ys[:, j])
        predicted_points_w_1[j].extend(pred[:, j])

R_square_values_w_1 = []

for point_idx in range(prompt_length):
    actual = torch.tensor(actual_points_w_1[point_idx])
    predicted = torch.tensor(predicted_points_w_1[point_idx])
    R_square = R_Square_Error(actual, predicted)
    R_square_values_w_1.append(R_square)

KeyboardInterrupt: 

In [ ]:
prompt_length = 2*conf.training.curriculum.points.end-1
print(prompt_length)

####  Part 2 - Covariate Shifts

##### W1 = 0.9, w2 = 0.1

In [ ]:
actual_points_w_9 = [[] for _ in range(prompt_length)]
predicted_points_w_9 = [[] for _ in range(prompt_length)]
# xs_list_w_9 = [] 
# ys_list_w_8 = [] 
for _ in tqdm(range(n_batches)):
    xs = data_sampler.sample_xs(b_size=batch_size, n_points=prompt_length, w1=0.9, w2=0.1)
    # xs_list_w_8.append(xs)
    ys = task.evaluate(xs)
    # ys_list_w_8.append(ys)

    with torch.no_grad():
        pred = model(xs, ys)
    for j in range(prompt_length):
        actual_points_w_9[j].extend(ys[:, j])
        predicted_points_w_9[j].extend(pred[:, j])
        
R_square_values_w_9 = []

for point_idx in range(prompt_length):
    actual = torch.tensor(actual_points_w_9[point_idx])
    predicted = torch.tensor(predicted_points_w_9[point_idx])
    R_square = R_Square_Error(actual, predicted)
    R_square_values_w_9.append(R_square)



##### W1 = 0.7, w2 = 0.3

In [ ]:
actual_points_w_7 = [[] for _ in range(prompt_length)]
predicted_points_w_7 = [[] for _ in range(prompt_length)]
# Generate data and perform the experiment
# xs_list_w_0 = [] 
# ys_list_w_0 = [] 
for _ in tqdm(range(n_batches)):
    xs = data_sampler.sample_xs(b_size=batch_size, n_points=prompt_length, w1=0.7, w2=0.3)
    # xs_list_w_0.append(xs)
    ys = task.evaluate(xs)
    # ys_list_w_0.append(ys)

    with torch.no_grad():
        pred = model(xs, ys)
    for j in range(prompt_length):
        actual_points_w_7[j].extend(ys[:, j])
        predicted_points_w_7[j].extend(pred[:, j])
        

R_square_values_w_7 = []

for point_idx in range(prompt_length):
    actual = torch.tensor(actual_points_w_7[point_idx])
    predicted = torch.tensor(predicted_points_w_7[point_idx])
    R_square = R_Square_Error(actual, predicted)
    R_square_values_w_7.append(R_square)



##### W1 = 0.5, w2 = 0.5

In [ ]:
actual_points_w_5 = [[] for _ in range(prompt_length)]
predicted_points_w_5 = [[] for _ in range(prompt_length)]
# xs_list_w_1 = [] 
# ys_list_w_1 = [] 
all_errors_w1_5 = []
for _ in tqdm(range(n_batches)):
    xs = data_sampler.sample_xs(b_size=batch_size, n_points=prompt_length, w1=0.5, w2=0.5)
    # xs_list_w_1.append(xs)
    ys = task.evaluate(xs)
    # ys_list_w_1.append(ys)

    with torch.no_grad():
        pred = model(xs, ys)
    for j in range(prompt_length):
        actual_points_w_5[j].extend(ys[:, j])
        predicted_points_w_5[j].extend(pred[:, j])
        

R_square_values_w_5 = []

for point_idx in range(prompt_length):
    actual = torch.tensor(actual_points_w_5[point_idx])
    predicted = torch.tensor(predicted_points_w_5[point_idx])
    R_square = R_Square_Error(actual, predicted)
    R_square_values_w_5.append(R_square)



In [ ]:

plt.figure(figsize=(10, 5), facecolor='none')
plt.plot(range(prompt_length), R_square_values_w_1, label='w1=1, w2=0', color='darkred', linewidth=2)
plt.plot(range(prompt_length), R_square_values_w_9, label='w1=0.9, w2=0.1', color='blue', linewidth=2)
plt.plot(range(prompt_length), R_square_values_w_7, label='w1=0.7, w2=0.3', linewidth=2)
plt.plot(range(prompt_length), R_square_values_w_5, label='w1=0.5, w2=0.5', linewidth=2)

plt.axhline(1, ls="--", color="darkgray", label="Baseline")
plt.axvline(x=(prompt_length-1)/2, color='grey', linestyle='--')  # Grey vertical line
# Adding legend, labels, and title
plt.legend(loc="upper right")
plt.xlabel('In-context Examples')
plt.ylabel('Mean Squared Errors')
plt.title('Comparison of RSE with Different w1 and w2')

plt.grid(color='lightgray', linestyle='-', linewidth=0.5)
# plt.savefig("../output/4.1_test_3_Sample_Selection_Covariate_Shifts.pdf", bbox_inches='tight', transparent=True)
# Show the plot
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [ ]:
# X_array = np.array(xs_list_w_9)
# y_array = np.array(ys_list_w_9)
# print("Shape of the first element in X_array:", X_array[0].shape)
# print("Shape of y_array:", y_array.shape)
# X_reshaped_list = [x_tensor.numpy().reshape(-1, 101*20) for x_tensor in X_array]
# y_reshaped_list = [y_tensor.numpy().reshape(-1, 101) for y_tensor in y_array]
# X_reshaped = np.concatenate(X_reshaped_list, axis=0)
# y_reshaped = np.concatenate(y_reshaped_list, axis=0)

In [ ]:
# # Splitting dataset into training and testing set
# X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_reshaped, test_size=0.5, random_state=42)

# # Creating Linear Regression model
# sklearn_linear_model = LinearRegression()

# # Training the model
# sklearn_linear_model.fit(X_train, y_train)

# # Making predictions
# y_pred = sklearn_linear_model.predict(X_test)

# # Evaluating the model
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)


#### 4.2 Sensitivity to Label Format: second part <strong>Linear regression with random labels<strong>

In [ ]:

actual_points_random = [[] for _ in range(prompt_length)]
predicted_points_random = [[] for _ in range(prompt_length)]

# Generate data and perform the experiment
for batch_idx in tqdm(range(50)):
    xs = xs_list[batch_idx]
    ys = ys_list[batch_idx]

    # j's idx starts at 1, but in graph, we refer it to 0
    for j in range(1, prompt_length+1):
        permuted_ys = np.copy(ys)
        if j > 2:  # if j == 1, 2, there are no prior labels or no need to permuted
            for i in range(batch_size):
                np.random.shuffle(permuted_ys[i, :j-1])
    
    # Transfer np array to tensor
    permuted_ys_tensor = torch.from_numpy(permuted_ys)
    # predict with the si
    with torch.no_grad():
        pred = model(xs, permuted_ys_tensor)
    
    for j in range(prompt_length):
        actual_points_random[j].extend(ys[:, j])
        predicted_points_random[j].extend(pred[:, j])

R_square_values_w_random = []

for point_idx in range(prompt_length):
    actual = torch.tensor(actual_points_random[point_idx])
    predicted = torch.tensor(predicted_points_random[point_idx])
    R_square = R_Square_Error(actual, predicted)
    R_square_values_w_random.append(R_square)

In [ ]:
# mean_errors_random_labels = np.mean(all_errors_random_labels, axis=(0, 2))

In [ ]:

plt.plot(range(prompt_length), R_square_values_w_1, label="True Labels",color='darkred', linewidth=2)
plt.plot(range(prompt_length), R_square_values_w_random, label="Random Labels", linewidth=2)
plt.axhline(1, ls="--", color="darkgray", label="Baseline", linewidth=2)
# existing_ticks = plt.xticks()[0]
# new_ticks = sorted(set(existing_ticks).union({10}))
# plt.xticks(new_ticks)
plt.legend()
plt.xlabel('In-context Examples')
plt.ylabel('Mean Squared Errors')
plt.title('MSE with Random Labels for Each In-context Examples')
plt.grid(color='lightgray', linestyle='-', linewidth=0.5)
# plt.savefig("../output/4.2_Std_Sensitivity_to_Label_Format.pdf", bbox_inches = "tight", transparent=True)
plt.show()